In [2]:
import os
import pprint
import tempfile

from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
import pandas as pd

In [3]:
spark = SparkSession.builder.appName("retrival").getOrCreate()

In [4]:
dataPath = "hdfs://localhost:9000/user/nhom7/book/data/"
bookFilePath = dataPath + "BX-Books.csv"
ratingsFilePath = dataPath + "BX-Book-Ratings.csv"
books_df = spark.read.options(inferSchema="true", header="true", delimiter=';').csv(bookFilePath)
rating_df = spark.read.options(inferSchema="true", header="true", delimiter=';').csv(ratingsFilePath)

In [5]:
rating_pd = rating_df.drop('Book-Rating').toPandas()
book_pd = books_df.select('ISBN').toPandas()

In [6]:
ratings = tf.data.Dataset.from_tensor_slices(dict(rating_pd))
books = tf.data.Dataset.from_tensor_slices(dict(book_pd))

In [7]:
ratings = ratings.map(lambda x: {
    "isbn": x["ISBN"],
    "user_id": x["User-ID"],
})

books = books.map(lambda x: x["ISBN"])

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [8]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [9]:
book_ISBN = books.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_book_ISBN = np.unique(np.concatenate(list(book_ISBN)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [10]:
embedding_dimension = 32

In [11]:
user_model = tf.keras.Sequential([
  tf.keras.layers.IntegerLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

In [12]:
book_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_book_ISBN, mask_token=None),
  tf.keras.layers.Embedding(len(unique_book_ISBN) + 1, embedding_dimension)
])

In [13]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=books.batch(128).map(book_model)
)

task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [14]:
class RetrivalModel(tfrs.Model):

  def __init__(self, user_model, book_model):
    super().__init__()
    self.book_model: tf.keras.Model = book_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_book_embeddings = self.book_model(features["isbn"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_book_embeddings)

In [15]:
model = RetrivalModel(user_model, book_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [16]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [17]:
model.fit(cached_train, epochs=3)

Epoch 1/3
10/10 [==============================] - 1417s 140s/step - factorized_top_k/top_1_categorical_accuracy: 0.0587 - factorized_top_k/top_5_categorical_accuracy: 0.0599 - factorized_top_k/top_10_categorical_accuracy: 0.0606 - factorized_top_k/top_50_categorical_accuracy: 0.0634 - factorized_top_k/top_100_categorical_accuracy: 0.0659 - loss: 69998.2507 - regularization_loss: 0.0000e+00 - total_loss: 69998.2507  
Epoch 2/3
10/10 [==============================] - 1263s 126s/step - factorized_top_k/top_1_categorical_accuracy: 0.0798 - factorized_top_k/top_5_categorical_accuracy: 0.0858 - factorized_top_k/top_10_categorical_accuracy: 0.0886 - factorized_top_k/top_50_categorical_accuracy: 0.1001 - factorized_top_k/top_100_categorical_accuracy: 0.1092 - loss: 68684.8509 - regularization_loss: 0.0000e+00 - total_loss: 68684.8509
Epoch 3/3
10/10 [==============================] - 1203s 120s/step - factorized_top_k/top_1_categorical_accuracy: 0.0640 - factorized_top_k/top_5_categorical_ac

In [18]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 355s 70s/step - factorized_top_k/top_1_categorical_accuracy: 0.0108 - factorized_top_k/top_5_categorical_accuracy: 0.0420 - factorized_top_k/top_10_categorical_accuracy: 0.0432 - factorized_top_k/top_50_categorical_accuracy: 0.0485 - factorized_top_k/top_100_categorical_accuracy: 0.0526 - loss: 32628.8073 - regularization_loss: 0.0000e+00 - total_loss: 32628.8073


{'factorized_top_k/top_1_categorical_accuracy': 0.01080000028014183,
 'factorized_top_k/top_5_categorical_accuracy': 0.041999999433755875,
 'factorized_top_k/top_10_categorical_accuracy': 0.04320000112056732,
 'factorized_top_k/top_50_categorical_accuracy': 0.04854999855160713,
 'factorized_top_k/top_100_categorical_accuracy': 0.05260000005364418,
 'loss': 29663.412109375,
 'regularization_loss': 0,
 'total_loss': 29663.412109375}

In [19]:
# Create a model that takes in raw query features, and
bruteforce = tfrs.layers.factorized_top_k.BruteForce(model.user_model, k=20)
# recommends movies out of the entire movies dataset.
bruteforce.index_from_dataset(
  tf.data.Dataset.zip((books.batch(128), books.batch(128).map(model.book_model)))
)

# Get recommendations.
_, titles = bruteforce(tf.constant([12]))
# 

for t in titles:
  print(t)

tf.Tensor(
[b'0552998486' b'0971880107' b'0099771519' b'8838918600' b'8807810778'
 b'880781210X' b'884590184X' b'0552996009' b'0743222229' b'8806142100'
 b'848300268X' b'0349101779' b'0312422156' b'0330244078' b'0446355569'
 b'0140071814' b'0590660543' b'1853260134' b'8432210900' b'0330331744'], shape=(20,), dtype=string)


In [20]:
path = ("./model/retrival_model/1/")
tf.saved_model.save(
    bruteforce,
    path
)

loaded = tf.saved_model.load(path)

INFO:tensorflow:Assets written to: ./model/retrival_model/1/assets


INFO:tensorflow:Assets written to: ./model/retrival_model/1/assets
